In [1]:
import numpy as np
import pandas as pd
import torch
from gcn_model import GCNModel
import utilities

In [2]:
data_folder = "simulations/splat_0.1_de_rq/"

In [3]:
# get labels
data_path = data_folder + "query_counts.csv"
tools = ["sctype","scsorter","scina","singler"]
ref_path = data_folder + "ref_counts.csv"
ref_label_path = data_folder + "ref_labels.csv"
marker_path = data_folder + "markers.txt"
all_labels = utilities.label_counts(data_path,tools,ref_path,ref_label_path,marker_path)

R[write to console]: Loading required package: Seurat




    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

R[write to console]: Attaching SeuratObject

R[write to console]: Attaching sp

R[write to console]: Loading required package: SCINA

R[write to console]: Loading required package: MASS

R[write to console]: Loading required package: gplots

R[write to console]: 
Attaching package: ‘gplots’


R[write to console]: The following object is masked from ‘package:stats’:

    lowess


R[write to console]: Loading required package: scSorter

R[write to console]: Loading required package: dplyr

R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following object is masked from ‘package:MASS’:

    select


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


R[write to console]: Loading required package: SingleR

R[write to console]: Loading required package: SummarizedExperiment

R[write to console]: Loading re

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 1000
Number of edges: 38730

Running Louvain algorithm...


R[write to console]: 0%   10   20   30   40   50   60   70   80   90   100%

R[write to console]: [----|----|----|----|----|----|----|----|----|----|

R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: 

Maximum modularity in 10 random starts: 0.4120
Number of communities: 7
Elapsed time: 0 seconds


R[write to console]: Performing log-normalization

R[write to console]: 0%   10   20   30   40   50   60   70   80   90   100%

R[write to console]: [----|----|----|----|----|----|----|----|----|----|

R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to conso

In [4]:
all_labels

,scina,scsorter,sctype,singler
Cell1001,Group1,Group4,None,Group1
Cell1002,Group1,Group1,None,Group4
Cell1003,Group2,Group2,None,Group2
Cell1004,Group2,Group2,None,Group2
Cell1005,Group2,Group3,None,Group2
...,...,...,...,...
Cell1996,Group3,Group4,None,Group3
Cell1997,Group1,Group1,None,Group1
Cell1998,Group4,Group3,None,Group4
Cell1999,Group1,Group1,None,Group1


In [5]:
_,marker_names = utilities.read_marker_file(marker_path)
marker_names

['Group1', 'Group2', 'Group3', 'Group4']

In [6]:
all_labels['sctype'].unique()

array([None, 'Group1'], dtype=object)

In [7]:
all_labels_factored = utilities.factorize_df(all_labels, marker_names)
encoded_labels = utilities.encode_predictions(all_labels_factored)
encoded_labels

array([[2., 0., 0., 1.],
       [2., 0., 0., 1.],
       [0., 3., 0., 0.],
       ...,
       [0., 0., 1., 2.],
       [3., 0., 0., 0.],
       [0., 1., 1., 1.]])

In [8]:
# read in dataset
X = pd.read_csv(data_path, index_col=0)
X = utilities.preprocess(np.array(X), scale=False)
X.shape

(1000, 500)

In [9]:
meta_path = data_folder + "query_meta.csv"
metadata = pd.read_csv(meta_path, index_col=0)
real_y = pd.factorize(metadata['Group'], sort=True)[0]
real_y.shape

(1000,)

In [10]:
print(utilities.pred_accuracy(all_labels_factored['scina'], real_y))
print(utilities.pred_accuracy(all_labels_factored['sctype'], real_y))
print(utilities.pred_accuracy(all_labels_factored['scsorter'], real_y))
print(utilities.pred_accuracy(all_labels_factored['singler'], real_y))


0.8669999837875366
0.0430000014603138
0.7139999866485596
0.5979999899864197


In [11]:
max_pred = torch.tensor(encoded_labels).max(dim=1)[1]
utilities.pred_accuracy(max_pred, real_y)

/Users/daniel/Desktop/conradLab/thesis/scSHARP/utilities.py:168: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return float((torch.tensor(preds) == torch.tensor(real)).type(torch.FloatTensor).mean().numpy())


0.8550000190734863

In [12]:
confident_labels = utilities.get_consensus_labels(encoded_labels, necessary_vote = 2)
confident_labels

array([ 0.,  0.,  1.,  1.,  1.,  2., -1.,  0.,  0.,  2.,  0.,  2.,  0.,
        0.,  0.,  1.,  3.,  3.,  0.,  1.,  2.,  1.,  0.,  3.,  0.,  1.,
       -1.,  3.,  3.,  0.,  2., -1.,  0.,  0.,  1.,  0.,  0.,  1.,  3.,
        1.,  2.,  0.,  1.,  1.,  1.,  0.,  0.,  0.,  3.,  0.,  2.,  0.,
        0.,  0.,  1.,  2.,  1.,  0.,  3.,  0.,  2.,  3.,  3.,  0.,  3.,
        1.,  0.,  0.,  3.,  2.,  0.,  0.,  1.,  0.,  2.,  0.,  0.,  0.,
        3., -1.,  2.,  0.,  0.,  2.,  0.,  1.,  0.,  1.,  1.,  1., -1.,
        3.,  1.,  1.,  0.,  3.,  3.,  2.,  0.,  0.,  0.,  0.,  2.,  0.,
       -1.,  2.,  1.,  2.,  2.,  3.,  0.,  0.,  1.,  1.,  0.,  3.,  0.,
        3.,  0.,  3.,  0.,  0.,  3.,  1.,  1.,  0.,  2.,  3.,  1.,  2.,
        0.,  0.,  2.,  1.,  2., -1.,  0.,  0.,  3.,  1.,  0.,  0.,  0.,
        2.,  0.,  0.,  1.,  0.,  0., -1.,  0.,  3.,  0.,  3.,  1.,  2.,
        0.,  3.,  0.,  0.,  2., -1.,  0.,  2.,  1.,  1.,  2.,  1.,  0.,
        1.,  0.,  0.,  0.,  0.,  3.,  1.,  1.,  2.,  2., -1.,  1

In [13]:
train_nodes = np.where(confident_labels != -1)[0]
test_nodes = np.where(confident_labels == -1)[0]
print(np.unique(confident_labels))
print(np.unique(confident_labels[train_nodes]))
print(np.unique(confident_labels[test_nodes]))

[-1.  0.  1.  2.  3.]
[0. 1. 2. 3.]
[-1.]


In [14]:
print(utilities.pred_accuracy(confident_labels[train_nodes], real_y[train_nodes]))

0.8796592354774475


In [15]:
# tool accuracy on test
print(utilities.pred_accuracy(np.array(all_labels_factored['scina'][test_nodes]), real_y[test_nodes]))
print(utilities.pred_accuracy(np.array(all_labels_factored['sctype'][test_nodes]), real_y[test_nodes]))
print(utilities.pred_accuracy(np.array(all_labels_factored['scsorter'][test_nodes]), real_y[test_nodes]))
print(utilities.pred_accuracy(np.array(all_labels_factored['singler'][test_nodes]), real_y[test_nodes]))

0.6721311211585999
0.0
0.1147540956735611
0.16393442451953888


In [16]:
dataset  = torch.utils.data.TensorDataset(torch.tensor(X), torch.tensor(confident_labels))
dataloader = torch.utils.data.DataLoader(dataset, batch_size=35, shuffle=True)

test_dataset  = torch.utils.data.TensorDataset(torch.tensor(X), torch.tensor(real_y))
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=35, shuffle=False)

In [17]:
m = GCNModel("configs/semi_basic_linear.txt", 2)

In [18]:
m.train(dataloader, 100)

Loss in epoch 0 = 40.510651
Loss in epoch 10 = 1.290930
Loss in epoch 20 = 0.288985
Loss in epoch 30 = 0.081738
Loss in epoch 40 = 0.083919
Loss in epoch 50 = 0.042423
Loss in epoch 60 = 0.015237
Loss in epoch 70 = 0.312704
Loss in epoch 80 = 0.105601
Loss in epoch 90 = 0.021684


In [19]:
m.validation_metrics(test_dataloader, train_nodes, test_nodes)

(0.8560000061988831,
 array([[372,  33,  11,  15],
        [ 14, 184,   4,  11],
        [ 18,   6, 195,  10],
        [  7,  12,   3, 105]]),
 0.8796592354774475,
 array([[354,  26,   8,  15],
        [ 10, 178,   3,   9],
        [ 12,   5, 190,   9],
        [  7,   8,   1, 104]]),
 0.49180328845977783,
 array([[18,  7,  3,  0],
        [ 4,  6,  1,  2],
        [ 6,  1,  5,  1],
        [ 0,  4,  2,  1]]))